In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
pd.options.display.max_rows = 999

# Generic Data Augmentation

# Baseline runs

In [2]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs   = [
            "MT_Baseline_e355dad2678c98182610abce1454bad8/Metrics"
        ]   

stats_filename = "agg_class_stats.pkl"

dhard_base = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets"

dhard_runs =  ["df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val_empirical_71d2180577457bea97ef33317560aeb8.pkl",
              ]

dhard_sub_runs = [run.replace("val_test_empirical", "val_test_sub_empirical") for run in dhard_runs]

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"
df_val = pd.read_pickle(val_path)


# Start analysis for entire val set
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy']['best'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)

df_base = df_val_classes.copy()

cols = [f"accuracy_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_base[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))   

thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_base[df_base['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [87.26666831970215]
Mean accuracy across runs :  87.26666831970215
Mean Acc for thrashed classes : 39.33333396911621


# Full

# Augmented runs

In [3]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_93d9fba92fe474a88dc763b34422a76c/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [96.44444580078125]
Mean accuracy across runs :  96.44444580078125
Mean Acc for thrashed classes : 85.5555534362793


# Analysis of DHard and DHard_sub

In [4]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [5]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,81.762924,100.0,86.666664
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,68.202766,100.0,84.444443


In [6]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 74.98284530639648
Mean dhard_sub performance 100.0
Mean acc_improvement 85.5555534362793


# RANDOM

# Augmented runs

In [7]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_0a6b144d32f8fe6abf86b09af4e8abfc/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [95.0000015258789]
Mean accuracy across runs :  95.0000015258789
Mean Acc for thrashed classes : 79.33333206176758


# Analysis of DHard and DHard_sub

In [8]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [9]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,79.027359,100.0,84.666664
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,53.917049,87.5,74.000000


In [10]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 66.47220420837402
Mean dhard_sub performance 93.75
Mean acc_improvement 79.33333206176758


In [ ]:
93.75

# UNICOM


# Augmented runs

In [2]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_97cef2f06183c96a24bd0ad36ada735a/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [91.13333511352539]
Mean accuracy across runs :  91.13333511352539
Mean Acc for thrashed classes : 58.333335876464844


# Analysis of DHard and DHard_sub

In [3]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [4]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,19.148937,50.0,39.555557
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,52.995392,100.0,77.111115


In [5]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 36.07216453552246
Mean dhard_sub performance 75.0
Mean acc_improvement 58.333335876464844


# GRADMATCH

# Augmented runs

In [2]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_0a6b144d32f8fe6abf86b09af4e8abfc/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [92.55555725097656]
Mean accuracy across runs :  92.55555725097656
Mean Acc for thrashed classes : 65.77777862548828


# Analysis of DHard and DHard_sub

In [3]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [4]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,34.954411,62.5,51.555557
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,58.986176,100.0,80.000000


In [5]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 46.970293045043945
Mean dhard_sub performance 81.25
Mean acc_improvement 65.77777862548828


# CORRUPTED LABELS

# Full

# Augmented runs

In [2]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_456c4c1f0ea8af6459d6f431baa5f935/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [92.0222240447998]
Mean accuracy across runs :  92.0222240447998
Mean Acc for thrashed classes : 63.00000190734863


# Analysis of DHard and DHard_sub

In [3]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [4]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,36.778118,87.5,52.666668
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,47.465439,87.5,73.333336


In [5]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 42.12177848815918
Mean dhard_sub performance 87.5
Mean acc_improvement 63.00000190734863


# RANDOM

# Augmented runs

In [7]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_e2d3c7f2e630591899b2ff1ac42bc00f/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [87.91111259460449]
Mean accuracy across runs :  87.91111259460449
Mean Acc for thrashed classes : 42.33333396911621


# Analysis of DHard and DHard_sub

In [8]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [9]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,6.686931,12.5,31.777779
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,8.294931,25.0,52.888889


In [10]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 7.490931034088135
Mean dhard_sub performance 18.75
Mean acc_improvement 42.33333396911621


# UNICOM


# Augmented runs

In [ ]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_97cef2f06183c96a24bd0ad36ada735a/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [91.13333511352539]
Mean accuracy across runs :  91.13333511352539
Mean Acc for thrashed classes : 58.333335876464844


# Analysis of DHard and DHard_sub

In [ ]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [ ]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,19.148937,50.0,39.555557
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,52.995392,100.0,77.111115


In [ ]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 36.07216453552246
Mean dhard_sub performance 75.0
Mean acc_improvement 58.333335876464844


# GRADMATCH

# Augmented runs

In [11]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_c0cdb19ad440713baf0a9498a954dad9/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [91.37777976989746]
Mean accuracy across runs :  91.37777976989746
Mean Acc for thrashed classes : 59.777780532836914


# Analysis of DHard and DHard_sub

In [12]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [13]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,28.875381,62.5,46.666668
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,45.161289,87.5,72.888893


In [14]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 37.01833534240723
Mean dhard_sub performance 75.0
Mean acc_improvement 59.777780532836914


# CORRUPTED DATA

In [10]:
import pickle
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
pd.options.display.max_rows = 999

# Full

# Augmented runs

In [2]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_5bb065f00602d5c9f9b87bb081cc8683/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [92.88889083862304]
Mean accuracy across runs :  92.88889083862304
Mean Acc for thrashed classes : 67.55555725097656


# Analysis of DHard and DHard_sub

In [3]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [4]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,46.200611,75.0,60.222221
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,48.847927,100.0,74.888893


In [5]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 47.524269104003906
Mean dhard_sub performance 87.5
Mean acc_improvement 67.55555725097656


# RANDOM

# Augmented runs

In [6]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_dcc18063f65ca13e4d6787f917227518/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [87.80000152587891]
Mean accuracy across runs :  87.80000152587891
Mean Acc for thrashed classes : 42.0


# Analysis of DHard and DHard_sub

In [7]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [8]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,7.902736,12.5,32.666668
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,5.069124,25.0,51.333332


In [9]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 6.4859299659729
Mean dhard_sub performance 18.75
Mean acc_improvement 42.0


# UNICOM


# Augmented runs

In [ ]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        ""
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [91.13333511352539]
Mean accuracy across runs :  91.13333511352539
Mean Acc for thrashed classes : 58.333335876464844


# Analysis of DHard and DHard_sub

In [ ]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [ ]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,19.148937,50.0,39.555557
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,52.995392,100.0,77.111115


In [ ]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 36.07216453552246
Mean dhard_sub performance 75.0
Mean acc_improvement 58.333335876464844


# GRADMATCH

# Augmented runs

In [11]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_2389d9924263c69d98908bbd8f5573d7/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [91.6444465637207]
Mean accuracy across runs :  91.6444465637207
Mean Acc for thrashed classes : 61.222225189208984


# Analysis of DHard and DHard_sub

In [12]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [13]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,29.179333,62.5,47.555557
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,49.308758,87.5,74.888893


In [14]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 39.24404525756836
Mean dhard_sub performance 75.0
Mean acc_improvement 61.222225189208984
